# Introduction à la classification d'images

Installation de librairies qui ne sont pas installées par défaut sur le service (déjà fait):
- `tqdm` : librairie pour afficher le progrès lorsqu'une boucle se trouve dans le code
- `torch` : une des librairies de référence pour le Deep Learning
- `torchvision` : librairie utilitaire mettant à disposition des données, des architectures de modèles, des fonctions de transformations d'images, etc.
- `pandas` : librairie de manipulations de données tabulaires
- `imageio` : librairie pour travailler avec des images
- `scikit-learn` : machine learning.

Avant d'importer les modules, il faut choisir la bonne image où sotn pré-installés les modules. Dans "Kernels", en bas à gauche, choisissez donc "application-image".

Import des modules nécessaires

In [5]:
%load_ext autoreload
%autoreload 2

import os
import s3fs
import shutil
import torch
from pathlib import Path
import random
import imageio.v3 as iio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


La cellule suivant indique si un ou plusieurs GPUs (Graphics Processing Units) sont disponibles et peuvent être utilisés pour faire les calculs.

In [6]:
if torch.cuda.is_available() : device = torch.device("cuda:0" )
else : device  = "cpu"
print("Using {} device".format(device))

if device != "cpu" :
    print("Nom du GPU :", torch.cuda.get_device_name(device=None))
    print("GPU initialisé : ", torch.cuda.is_initialized())

Using cpu device


## Chargement des données 

On récupère des images et fichiers utiles sur un espace de stockage distant.

In [20]:
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://minio.lab.sspcloud.fr'})
if not os.path.exists('oiseaux'):
    os.mkdir('oiseaux')
try:
    fs.get('projet-funathon/2022/diffusion/Sujet9_deep_learning_donnees_satellites/archive.zip', 'oiseaux/oiseau.zip')
except:
    !curl 'https://minio.lab.sspcloud.fr/projet-funathon/2022/diffusion/Sujet9_deep_learning_donnees_satellites/archive.zip' --output 'oiseaux/oiseau.zip'
shutil.unpack_archive('oiseaux/oiseau.zip', extract_dir='oiseaux')
os.remove('oiseaux/oiseau.zip')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1530M  100 1530M    0     0   121M      0  0:00:12  0:00:12 --:--:--  118M


OSError: [Errno 28] No space left on device

On souhaite faire de la classification d'images. Dans un premier temps, on se limite à un problème à 2 classes. On veut disposer d'un algorithme permettant de classer correctement des images de 2 espèces d'oiseaux (dans un premier temps).

Dans la cellule suivante, on importe une table `birds.csv` qui contient des chemins vers des images d'oiseaux ainsi que les espèces correspondantes associées. On crée un array `train_images_paths` qui contient l'ensemble des chemins permettant d'accéder aux images des classes `0` et `1` (respectivement *abbotts babbler* et *abbotts booby*). L'array `train_images_labels` contient les annotations des images.

Le dictionnaire `dic_id_to_label` contient la correspondance entre les noms des espèces les identifiants associés. Ici, 366 images d'entraînement sont à notre disposition.

In [9]:
bird_df = pd.read_csv('oiseaux/birds.csv')

NB_CLASSES = 2
# On se limite à 2 classes pour le moment
bird_df = bird_df[bird_df['class index'] < NB_CLASSES]

train_images_paths = np.array(['oiseaux/' + path for path in bird_df['filepaths']])
train_images_labels = np.array(bird_df['class index'])

# Création d'un dictionnaire des annotations
nom_oiseau = np.array(bird_df.labels.unique())
dic_id_to_label = {i : oiseau for i, oiseau in zip(range(NB_CLASSES), nom_oiseau)}
dic_id_to_label

{0: 'ABBOTTS BABBLER', 1: 'ABBOTTS BOOBY'}

In [10]:
print(train_images_labels.shape)
print(train_images_paths.shape)

(366,)
(366,)


## Création d'une classe `Dataset`

On crée ici une classe `CustomDataset` qui hérite de la classe `Dataset`, une classe utilitaire de librairie `Pytorch` qu'on initialise avec nos données et qui permet ensuite d'accéder facilement aux observations. Un `CustomDataset` sera initialisé avec l'ensemble des chemins vers les images et des annotations correspondantes.

Un objet `Dataset` est *itérable* dans la mesure où il est possible boucler dessus pour en récupérer les éléments. La méthode `_getitem_` permet justement d'accéder à un élément à partir d'un indice. Ci-dessous on demande à chaque itération de retourner un dictionnnaire dont les items (clef, valeur) sont : 
- (`image`, l'image sous la forme d'un `np.array`) 
- (`label`, l'annotation associée (0 ou 1)) 

In [11]:
class CustomDataset(Dataset):
    def __init__(self, image_paths,labels):
        self.image_paths = image_paths
        self.labels = labels

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = iio.imread(self.image_paths[idx])
        image = torch.tensor(np.array(image,dtype = float)/255, dtype=torch.float).permute(2,1,0)
        label = self.labels[idx]
        return {"image": image, "label" : label}

    def __len__(self):  # return count of sample we have
        return len(self.image_paths)

## Définition de quelques hyper-paramètres et paramètres d'entraînement

On fixe la valeurs de plusieurs paramètres sur lesquels on reviendra plus tard.

In [12]:
config = {
    'n_epoch' : 10,
    'val_size' : 0.15,
    'batch_size' : 30,
    'optimizer' : "SGD",
    'lr' : 0.005,
    'momentum' : 0.9,
    'model type': "convnet",
    'descriptif': "Entrainement avec un convnet"
}

## Création des `DataLoader`

Das un premier temps on va découper notre jeu d'apprentissage (les 366 images labellisées) en : 
- un jeu de d'entraînement sur lequel on entraînera notre modèle de classification
- un jeu de validation ne participant pas à l'entrainement du modèle mais permettant d'en évaluer les performances

La fonction `train_test_split` de la librairie `scikit-learn` permet de faire ce découpage. Le paramètre `val_size` du dictionnaire de configuration donne le taux d'images du jeu de données initial que l'on veut inclure dans le jeu de validation. Le paramètre `stratify` de la fonction `train_test_split` permet de préciser sur quel critère on veut stratifier cette sélection aléatoire (on choisit la variable de labels dans le but d'avoir un échantillon représentatif des labels de l'ensemble des images).

Lors de l'entraînement, on va procéder par itérations sur des petits paquets d'images (appelés *batchs*). Pour chaque batch, on calcule batchs l'erreur de prédiction et son gradient, puis on modifie les paramètres du modèle en conséquence (par descente de gradient).

L'extraction de paquets d'images est facilitée par la création d'objets `Dataloader`. Le paramètre `batch_size`permet de préciser la taille des batchs désirée. Le paramètre `shuffle` égal à `True` signifie que les images seront toutes remélangées une fois un tour total de l'ensemble des images réalisé. On appelle un tour complet une *epoch*.

In [13]:
batch_size =  config['batch_size']
all_dataset = CustomDataset(train_images_paths,train_images_labels)

# On découpe le data set en train et validation (20 % de validation) en stratifiant par les labels
train_indices, valid_indices = train_test_split(
    list(range(len(train_images_labels))),
    test_size=config['val_size'],
    stratify=train_images_labels
)

train_dataset = torch.utils.data.Subset(all_dataset, train_indices)
valid_dataset = torch.utils.data.Subset(all_dataset, valid_indices)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=80, shuffle=True, num_workers=0)

## Définition des modèles de classification

Les réseaux de neurones convolutionnels sont en général très efficaces pour des tâches de classification d'images. On définit ici une classe `Net` (héritant de la classe `nn.Module` de `Pytorch`) qui implémente un réseau de neurones convolutifs.

La méthode `__init__` permet de définir l'ensemble des couches utilisées dans la méthode de propagation `forward`. Cette méthode définit la suite des opérations appliquées aux images `x` passant par dans réseau `Net`.

In [14]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 4)
        self.conv4 = nn.Conv2d(32, 16, 3)
        self.conv5 = nn.Conv2d(16, 24, 3)
        self.fc1 = nn.Linear(24 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, NB_CLASSES)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

On a défini un modèle de réseau de neurones convolutifs. Cette structure est très utilisée lorsqu'on traite des problèmes sur les images. De manière assez générale en Machine Learning, on fonctionne souvent en 2 étapes : 
1) extraction de caractéristiques de l'entrée (extraction des composantes principales d'une ACP par exemple)
2) un classifieur est appliqué aux caractéristiques extraites, ce qui permet de prédire une classe

Les réseaux de neurones convolutifs fonctionnent sur le même principe, si ce n'est que l'extraction des caractéristiques est complètement internalisée dans le modèle. On apprend au modèle à extraire ces caractéristiques lui-même. L'extraction de caractéristique est permise par l'usage d'opérations de *convolution* et de *pooling* permettant de réduire la dimension de l'image.

Dans un premier temps (à l'intérieur du constructeur), on a défini les différents objets utiles à la construction du réseau.

Les couches `Conv2d` sont des couches *convolutives* appliquables aux images. Appliquer une convolution revient à appliquer un filtre sur l'image. Un filtre est une matrice réelle généralement de petite dimension. Appliquer ce filtre au pixel $(i,j)$ revient à effectuer la moyenne pondérée par les coefficients du filtre des pixels au voisinage de $(i,j)$, le voisinage étant défini par la dimension du filtre.

Les couches `Conv2D` prennent notamment en paramètres : 
- le nombre de canaux en entrée : la première convolution prend 3 canaux en entrée car c'est elle qui sera appliquée à notre image de départ qui a trois canaux (image en couleurs)
- le nombre de canaux en sortie : la première couche convolutive aura 6 canaux de sortie. Ce qui veut dire que 6 filtres seront générée par cette couche donnant naissance à 6 images en sortie résultantes de l'application des 6 filtres sur l'image 3 x 256 x 256 initiale
- la taille des filtres appliqués

Plein d'autres paramètres existent, que l'on peut consulter sur la documentation `Pytorch` (voir `stride` et `padding` par exemple). Les coefficients contenus dans les filtres font partie des paramètres du modèle, ce sont eux qui seront entraînés, *i.e* modifiés à chaque itération de l'entraînement.

L'opération *maxpool* est également une opération de réduction de la dimension, mais elle ne contient pas de paramètres. Le *maxpool* applique un filtre à une image en entrée.  Appliquer ce filtre au pixel $(i,j)$ revient à rechercher le coefficient maximum au voisinage de $(i,j)$, le voisinage étant défini par la dimension du filtre. 

L'extraction de caractéristiques de l'image est en fait réalisé par l'enchaînement de couches convolutives. Une couche convolutive entière est composée d'une couche `Conv2D`, d'une couche `Maxpool2D` et de l'application d'une fonction non linéaire (non paramétrée, ici la fonction ReLU). L'entraînement modifiera les paramètres de cet enchaînement de couches (en l'occurrence les paramètres des filtres de convolution) de telle sorte que l'extraction de caractéristiques soit la plus pertinente possible au regard du problème posé.

Après passage des 5 couches convolutives sur l'image initiale, 24 * 4 * 4 = 384 caractéristiques on été extraites. 
Les couches `Linear` permettent de construire le classifieur permettant par la suite de trancher entre le label 0, ou 1 pour l'image considérée. On notera par ailleurs que la sortie du réseau à la même dimension que le nombre de classes. Cette sortie de dimension 2 représente en fait le score attribué à telle ou telle classe par le modèle permettant de décider si l'image appartient à la classe 0 ou la classe 1.

## Initialisation du modèle

On crée un modèle de la classe `Net`.

In [15]:
net = Net()

On peut voir la structure de notre réseau grâce à la ligne suivante.

In [16]:
net

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(4, 4), stride=(1, 1))
  (conv4): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=384, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)

La fonction ci-dessous permet d'obtenir le nombre de paramètres de notre modèle.
Notre *petit* modèle contient en fait 75374 paramètres, ce qui dépasse l'ordre de grandeur usuel du nombre de paramètres rencontré en Machine Learning *classique*.

In [17]:
def get_n_params(model):
    pp = 0
    for p in list(model.parameters()):
        nn = 1
        for s in list(p.size()):
            nn = nn * s
        pp += nn
    return pp

get_n_params(net)

75734

## Visualisation des images

Ci-dessous une fonction permettant d'afficher une image d'oiseau.

In [18]:
def display_image(image):
    img = image.permute(2,1,0)
    img = np.array(img)
    plt.imshow(np.array(img))
    plt.show()

Ci-dessous on utilise la fonction précédente pour afficher des images du jeu d'entraînement aléatoirement en itérant sur `DataLoader` d'entraînement.

In [19]:
t = iter(train_loader)
dico = next(t)
images = dico["image"]
labels = dico["label"]

FileNotFoundError: No such file: '/home/onyxia/work/cours-nouvelles-donnees-site/applications/images/oiseaux/valid/ABBOTTS BABBLER/3.jpg'

In [ ]:
for i in range(5):
    print(labels[i])
    print(dic_id_to_label[int(labels[i])])
    display_image(images[i])

## Entrainement du modèle

Comme dit précédemment, entraîner le modèle revient à modifier, itération par itération ses paramètres afin que ce dernier soit le plus pertinent possible dans la tâche de classification qu'il doit réaliser. Une itération consiste en 4 étapes :

1) Récupération d'un batch d'images
2) Calcul de l'erreur de classification commise par le modèle sur le batch (l'erreur est une fonction de ses paramètres)
3) Calcul du gradient de l'erreur
4) Descente du gradient, *i.e* on bouge les paramètres dans la direction opposée du gradient calculé précédemment pour abaisser l'erreur commise par le modèle. Le taux d'apprentissage permet de définir l'amplitude du mouvement dans cette direction

La structure des réseaux de neurones est telle que les calculs peuvent être effectués par des GPUS pour réduire grandement le temps de calcul. Comparer la différence de temps d'execution en initiant la variable device à "cuda:0" (ce qui veut dire que les calculs seront lancés sur GPU). Pour cela il faut que le service utilisé mette des GPUs à disposition.

In [ ]:
if torch.cuda.is_available(): device = torch.device("cuda:0" )
device

In [ ]:
# device = "cpu"
# device

**Lancement de l'entraînement**

On définit un processus d'optimisation `Adam` et une fonction de perte `CrossEntropyLoss` pour pouvoir procéder à l'entraînement.

In [ ]:
# optimizer = optim.SGD(net.parameters(), lr=config['lr'], momentum=config['momentum'])
optimizer = optim.Adam(net.parameters(), lr=config['lr'])
net = net.to(device)

entropy = nn.CrossEntropyLoss()

liste_loss = []
liste_acc_val = []
for epoch in range(config['n_epoch']):
    running_loss = 0.0

    t = tqdm(train_loader, desc="epoch %i" % (epoch+1), position = 0, leave=True)
    epoch_loop = enumerate(t)

    for i, data in epoch_loop:
        taille_batch = data['image'].shape[0]
        images = data['image']
        labels  =  data['label']
        images, labels = images.to(device), labels.to(device)
        pred = net(images)

        # Zero the parameter gradients
        optimizer.zero_grad()
        loss = entropy(pred,labels)
        loss.backward()  # Calculer le gradient
        optimizer.step()  # Avancer dans le sens du gradient calculé

        del images, labels, pred
        running_loss += loss.item()

        if (i+1) % 10 == 0:
            # Enregistrement de la loss sur le train, sur le validation
            liste_loss.append(running_loss)

            # Validation
            ech_val = iter(valid_loader)
            dico = next(ech_val)
            images = dico["image"]
            labels = dico["label"]

            pred = torch.argmax(net(images.to(device)), dim=1)
            labels = labels.to(device)

            acc_val = round(
                100 * int(torch.sum(pred == labels).cpu()) / int(np.array(labels.size())),
                2
            )
            liste_acc_val.append(acc_val)
            t.set_description("epoch %i, 'mean loss: %.6f', 'acc.val: %.2f'" % (epoch+1, running_loss/10, acc_val))
            t.refresh()
            running_loss =0

**Attention** : selon la configuration il faut parfois beaucoup d'epochs pour voir la perte diminuer, ne désespérez pas ! 
Si la perte ne descend pas, une option est de tester un autre optimiseur.

Si l'entraînement avance trop lentement, réouvrez un autre service et essayez de mettre en place la partie **Un autre jeu d'images pour s'amuser**, voir ci-dessous.

## Evaluation du réseau entraîné

Lors de l'entraînement, les moyennes de la perte sur le jeu d'entraînement et de la précision obtenue sur le jeu de validation sont enregistrées tous les 5 batchs dans :
- `liste_loss` 
- `liste_acc_val` 

On peut les représenter graphiquement.

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('epoch')
ax1.set_ylabel('valiadation accuracy', color=color)
ax1.plot(liste_acc_val, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()   # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('entropy loss', color=color)  # we already handled the x-label with ax1
ax2.plot(liste_loss, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

On constate que rapidement, la précision obtenue sur le jeu de validation se stabilise et même dimininue (sur-apprentissage). On pourrait arrêter l'entraînement à ce moment là.

On peut calculer la matrice de confusion pour le jeu de validation.

In [ ]:
t = iter(valid_loader)
dico = next(t)

images = dico["image"]
labels = dico["label"]

pred = np.array(torch.argmax(net(images.to(device)),dim =1).cpu())
labels = np.array(labels)

confusion_matrix(pred, labels)

On constate que (remplir les **??**) : 
- La majorité des images du jeu de validation sont correctement classées par notre algorithme
- **??** images du jeu de validation sont classées parmi les 0 par notre algorithme à tort
- **??** images du jeu de validation sont classées parmi les 1 par notre algorithme à tort

## Que faire pour continuer ?

Jouer sur les différents hyperparamètres (dans `config`). Quels sont les impacts sur les résultats du :
- Nombre d'élements par batch ?
- Nombre d'epochs ?
- Learning rate ?
- Momentum ?

Que se passe-t-il si on augmente le nombre de classes d'oiseau à prédire ? (`NB_CLASSES`)

## Un autre jeu d'images pour s'amuser

Ici on va charger un autre jeu d'images, de chats et de chiens.

In [ ]:
fs.get('projet-funathon/diffusion/Sujet9_deep_learning_donnees_satellites/train.zip', 'chat_chien.zip')
if not os.path.exists('chat_chien'):
    os.mkdir('chat_chien')
shutil.unpack_archive('chat_chien.zip', extract_dir='chat_chien')
os.remove('chat_chien.zip')

In [ ]:
dic_label_to_id  = {"cat": 0, "dog": 1}
train_images_file = os.listdir("chat_chien/train")

train_images_paths = ["chat_chien/train/" + path for path in train_images_file]
train_images_labels = [dic_label_to_id[st[0:3]] for st in train_images_file]

La liste des chemins de fichiers pointant vers les images et la liste des labels ayant été définies, vous pouvez essayer de reproduire les différentes étapes introduites précédemment :
1) Création de la classe dataset
2) Création des loaders (essayer d'afficher les images par la suite)
3) Création du modèle
4) Entraînement du modèle
5) Evaluation

## Documentation

- [Documentation Pytorch](https://pytorch.org/docs/stable/index.html)  
- [Un autre exemple de classification avec `Pytorch` (ici le data set KMNIST)](https://pyimagesearch.com/2021/07/19/pytorch-training-your-first-convolutional-neural-network-cnn/)  
- [Utiliser des réseaux pré-entrainés](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html) 